# Ejemplo API sencilla

Pagina APIS publicas: https://github.com/public-apis/public-apis#finance

Primer ejemplo: econdb

# Consumer Price index

In [1]:
import pandas as pd

API_TOKEN = '97c58ad355a11b2f37569e6c27e07a0fb88fa640'

df = pd.read_csv(
    'https://www.econdb.com/api/series/URATEUS/?token=%s&format=csv' % API_TOKEN,
    index_col='Date', parse_dates=['Date'])

In [45]:
import pandas as pd

API_TOKEN = '9ef3a111f8a84eaf8fd190d66ff6f2dc271bf766'

df = pd.read_csv(
    'https://www.econdb.com/api/series/CPIUS/?token=%s&format=csv' % API_TOKEN,
    index_col='Date', parse_dates=['Date'])
df.head()

,CPIUS
Date,
1947-01-01,21.48
1947-02-01,21.62
1947-03-01,22.00
1947-04-01,22.00
1947-05-01,21.95


In [2]:
df.tail()

,URATEUS
Date,
2022-11-01,3.6
2022-12-01,3.5
2023-01-01,3.4
2023-02-01,3.6
2023-03-01,3.5


In [3]:
df.shape

(903, 1)

In [4]:
df=df.reset_index()
df.head()

,Date,URATEUS
0,1948-01-01,3.4
1,1948-02-01,3.8
2,1948-03-01,4.0
3,1948-04-01,3.9
4,1948-05-01,3.5


Crear en conexion de destino:

```SQL
create table cpi(
Date date,
CPIUS float
);
```

In [58]:
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws"
data_base="data-engineer-database"
user="dafbustosus_coderhouse"
with open("C:/Users/Windows/Downloads/pwd_coder.txt",'r') as f:
    pwd= f.read()

In [59]:
import psycopg2
try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Connected to Redshift successfully!")
    
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

Connected to Redshift successfully!


In [52]:
cur = conn.cursor()
# Execute a SQL query to select data from a table
cur.execute("SELECT * FROM cpi")
# Fetch the results
results = cur.fetchall()
results

[]

In [53]:
df.head()

,Date,CPIUS
0,1947-01-01,21.48
1,1947-02-01,21.62
2,1947-03-01,22.00
3,1947-04-01,22.00
4,1947-05-01,21.95


In [54]:
from psycopg2.extras import execute_values
cur = conn.cursor()
# Define the table name
table_name = 'cpi'
# Define the columns you want to insert data into
columns = ['Date', 'CPIUS']
# Generate 
values = [tuple(x) for x in df.to_numpy()]
insert_sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES %s"
# Execute the INSERT statement using execute_values
cur.execute("BEGIN")
execute_values(cur, insert_sql, values)
cur.execute("COMMIT")
# Close the cursor and connection
#cur.close()
#conn.close()

# Ejemplo API - transformacion

messari API: https://messari.io/api

In [1]:
import requests
requests.get?

Signature: requests.get(url, params=None, **kwargs)
Docstring:
Sends a GET request.

:param url: URL for the new :class:`Request` object.
:param params: (optional) Dictionary, list of tuples or bytes to send
    in the query string for the :class:`Request`.
:param \*\*kwargs: Optional arguments that ``request`` takes.
:return: :class:`Response <Response>` object
:rtype: requests.Response
File:      c:\users\windows\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\site-packages\requests\api.py
Type:      function


In [2]:
import requests
import pandas as pd

url = "https://data.messari.io/api/v1/assets/bitcoin/metrics"
headers = {"Accept-Encoding": "gzip, deflate"}

response = requests.get(url, headers=headers)
data = response.json()

#df = pd.DataFrame.from_dict(data['data']['market_data'], orient='index')
#print(df)

El `Accept-Encoding` header se usa por el cliente (en el caso de Python) para indicar los formatos de codificación que puede entender en la respuesta. Luego, el servidor puede comprimir la respuesta utilizando uno de los formatos enumerados antes de enviarla de vuelta al cliente, lo que puede ahorrar ancho de banda y mejorar el rendimiento.

En este caso, "gzip" y "deflate" son dos formatos de compresión populares que el cliente indica que puede manejar. El servidor puede optar por utilizar cualquiera de los formatos para comprimir la respuesta si los admite.

In [ ]:
data

In [4]:
data.keys()

dict_keys(['status', 'data'])

In [5]:
data['status']['timestamp']

'2023-05-23T23:43:04.936866438Z'

In [6]:
datax = data['data']

In [ ]:
datax

In [10]:
import requests
url = "https://data.messari.io/api/v1/assets/bitcoin/metrics"
headers = {"Accept-Encoding": "gzip, deflate"}
response = requests.get(url, headers=headers)
response

<Response [200]>

In [9]:
datax = data['data']
dt = pd.DataFrame.from_dict(datax, orient='index')
dt

,0
id,1e31218a-e44e-4285-820c-8282ee222035
serial_id,6057
symbol,BTC
name,Bitcoin
slug,bitcoin
contract_addresses,None
_internal_temp_agora_id,9793eae6-f374-46b4-8764-c2d224429791
market_data,"{'price_usd': 27200.835514165432, 'price_btc':..."
marketcap,"{'rank': 1, 'marketcap_dominance_percent': 45...."
supply,"{'y_2050': 20986335.65, 'y_plus10': 20727672.4..."


In [10]:
dt.loc['mining_stats'][0]

{'mining_algo': 'SHA-256',
 'network_hash_rate': '389770 PH/s',
 'available_on_nicehash_percent': 0.03159645632061036,
 '1_hour_attack_cost': None,
 '24_hours_attack_cost': None,
 'attack_appeal': None,
 'hash_rate': 354690591.130938,
 'hash_rate_30d_average': 350197391.26665026,
 'mining_revenue_per_hash_usd': 8.50742e-07,
 'mining_revenue_per_hash_native_units': 3.2e-11,
 'mining_revenue_per_hash_per_second_usd': 0.073504145346,
 'mining_revenue_per_hash_per_second_native_units': 2.736694e-06,
 'mining_revenue_from_fees_percent_last_24_hours': 7.281457362,
 'mining_revenue_native': 970.67962286,
 'mining_revenue_usd': 26071228.76349346,
 'mining_revenue_total': 50869797144.95894,
 'average_difficulty': 49549703178592.68}

In [11]:
extract=dt.loc['mining_stats'][0]
e = pd.DataFrame.from_dict(extract, orient='index', columns=['value']).transpose().reset_index(drop=True)
e['Date'] = data['status']['timestamp']
e

,mining_algo,network_hash_rate,available_on_nicehash_percent,1_hour_attack_cost,24_hours_attack_cost,attack_appeal,hash_rate,hash_rate_30d_average,mining_revenue_per_hash_usd,mining_revenue_per_hash_native_units,mining_revenue_per_hash_per_second_usd,mining_revenue_per_hash_per_second_native_units,mining_revenue_from_fees_percent_last_24_hours,mining_revenue_native,mining_revenue_usd,mining_revenue_total,average_difficulty,Date
0,SHA-256,389770 PH/s,0.031596,None,None,None,354690591.130938,350197391.26665,0.000001,0.0,0.073504,0.000003,7.281457,970.679623,26071228.763493,50869797144.958939,49549703178592.679688,2023-05-23T23:43:04.936866438Z


In [12]:
print(e.columns)

Index(['mining_algo', 'network_hash_rate', 'available_on_nicehash_percent',
       '1_hour_attack_cost', '24_hours_attack_cost', 'attack_appeal',
       'hash_rate', 'hash_rate_30d_average', 'mining_revenue_per_hash_usd',
       'mining_revenue_per_hash_native_units',
       'mining_revenue_per_hash_per_second_usd',
       'mining_revenue_per_hash_per_second_native_units',
       'mining_revenue_from_fees_percent_last_24_hours',
       'mining_revenue_native', 'mining_revenue_usd', 'mining_revenue_total',
       'average_difficulty', 'Date'],
      dtype='object')


```SQL
CREATE TABLE mining_data (
  mining_algo VARCHAR(20),
  network_hash_rate VARCHAR(20),
  available_on_nicehash_percent FLOAT,
  one_hour_attack_cost FLOAT,
  twenty_four_hours_attack_cost FLOAT,
  attack_appeal FLOAT,
  hash_rate FLOAT,
  hash_rate_30d_average FLOAT,
  mining_revenue_per_hash_usd FLOAT,
  mining_revenue_per_hash_native_units FLOAT,
  mining_revenue_per_hash_per_second_usd FLOAT,
  mining_revenue_per_hash_per_second_native_units FLOAT,
  mining_revenue_from_fees_percent_last_24_hours FLOAT,
  mining_revenue_native FLOAT,
  mining_revenue_usd FLOAT,
  mining_revenue_total FLOAT,
  average_difficulty FLOAT,
  Date TIMESTAMP
);
```

In [65]:
cur = conn.cursor()
# Execute a SQL query to select data from a table
cur.execute("SELECT * FROM mining_data")
# Fetch the results
results = cur.fetchall()
results

[]

In [66]:
e.head()

,mining_algo,network_hash_rate,available_on_nicehash_percent,1_hour_attack_cost,24_hours_attack_cost,attack_appeal,hash_rate,hash_rate_30d_average,mining_revenue_per_hash_usd,mining_revenue_per_hash_native_units,mining_revenue_per_hash_per_second_usd,mining_revenue_per_hash_per_second_native_units,mining_revenue_from_fees_percent_last_24_hours,mining_revenue_native,mining_revenue_usd,mining_revenue_total,average_difficulty,Date
0,SHA-256,334978 PH/s,0.043239,None,None,None,323675130.890479,332859685.871674,0.000001,0.0,0.078492,0.000003,3.603432,901.22503,25405814.413858,49531696788.625618,46843400286276.546875,2023-04-07T17:36:49.198027238Z


In [67]:
columns= ['mining_algo', 'network_hash_rate', 'available_on_nicehash_percent',
'one_hour_attack_cost', 'twenty_four_hours_attack_cost', 'attack_appeal',
'hash_rate', 'hash_rate_30d_average', 'mining_revenue_per_hash_usd',
'mining_revenue_per_hash_native_units',
'mining_revenue_per_hash_per_second_usd',
'mining_revenue_per_hash_per_second_native_units',
'mining_revenue_from_fees_percent_last_24_hours',
'mining_revenue_native', 'mining_revenue_usd', 'mining_revenue_total',
'average_difficulty', 'Date']

In [68]:
from psycopg2.extras import execute_values
cur = conn.cursor()
# Define the table name
table_name = 'mining_data'
# Define the columns you want to insert data into
columns = columns
# Generate 
values = [tuple(x) for x in e.to_numpy()]
insert_sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES %s"
# Execute the INSERT statement using execute_values
cur.execute("BEGIN")
execute_values(cur, insert_sql, values)
cur.execute("COMMIT")
# Close the cursor and connection
#cur.close()
#conn.close()

In [43]:
# Si hay algun error volver de la transaccion (OJO: solo usar cuando haya un error en la transaccion)
conn.rollback()

In [69]:
cur.close()

In [70]:
conn.close()